In [1]:
import DWM1_Tokenizer
import DWM2_Global_Token_Replace_NewDict
import DWM3_Entropy_Cluster
# Settings for Tokenizer
inputSampleName = r'S9'
delimiter=','
hasHeader = True
tokenizerType = 'Splitter'
removeDuplicateTokens = True

# Setting for global replacement DWM2
runReplacement = True
minFreqStdToken = 5
minLenStdToken = 3
maxFreqErrToken = 3

# Cluster Settings


DWM1_Tokenizer.tokenizeCycle(inputSampleName, delimiter, hasHeader, removeDuplicateTokens, tokenizerType)
prefixString = inputSampleName +','+tokenizerType+','+str(removeDuplicateTokens)
fileToCluster = inputSampleName + '-Tokenized.txt'
if runReplacement:
    DWM2_Global_Token_Replace_NewDict.globalReplace(inputSampleName, minFreqStdToken, minLenStdToken, maxFreqErrToken)
    fileToCluster = inputSampleName + '-TokenReplace.txt'
    prefixString = prefixString+','+str(minFreqStdToken)+','+str(minLenStdToken)+','+str(maxFreqErrToken)
else:
    prefixString = prefixString+',0,0,0'

# Cluster fixed settings
epsilonIterate = 0.0
runClusterMetrics = False
runFinalMetrics = True
createFinalJoin = False

Tokenizing References
Total References= 1000
Total Tokens Found = 12782
Total Tokens Output = 12731
Dictionary Load = 67077
<class 'dict'>
Global Replace Cycle
Total References= 1000
Total Tokens= 12731
Unique Tokens= 3675
Minimum Frequency of Standard Token =  5
Minimum Length of Standard Token =  3
Maximum Frequency of Error Token =  3
References Processed= 1000
Sorted Token Size = 3675
Clean Token Size = 1694
*Stop Replacements here
References Processed =  1000
Total Replacement Pairs = 212
Tokens Read = 12731
Tokens Changed =  224
References Changed = 204


In [2]:
status="done"

In [3]:
print(status)

done


In [14]:
# # Cluster Robot Settings
# muStart = 0.60
# muEnd = 0.60
# muIncr = 0.05
# betaStart = 14
# betaEnd = 14
# betaIncr = 1
# sigmaStart = 145
# sigmaEnd = 145
# sigmaIncr = 1
# epsilonStart = 35
# epsilonEnd = 35
# epsilonIncr = 1
# mu = muStart
# while mu <= muEnd:
#     beta = betaStart
#     while beta <= betaEnd:
#         sigma = sigmaStart
#         while sigma <= sigmaEnd:
#             epsilon = epsilonStart
#             while epsilon <= epsilonEnd:
#                 suffixString =','+str(epsilon)+','+str(epsilonIterate)
#                 resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, beta, sigma, mu, muIncr,epsilon,  \
#                                                      epsilonIterate,runClusterMetrics, runFinalMetrics, createFinalJoin, hasHeader)
#                 seg0a = ','+str(resultsList[0])+','+str(resultsList[1])+','+str(resultsList[2])+','+str(resultsList[3])
#                 seg0b =','+str(resultsList[4])+','+str(resultsList[5])+','+str(resultsList[6])
#                 seg1 = ','+str(beta)+','+str(sigma)+','+str(mu)+ ',' +str(muIncr)
#                 seg2 = ','+str(resultsList[7])+','+str(resultsList[8])+','+str(resultsList[9])
#                 outString = prefixString+seg0a+seg0b+suffixString+seg1+seg2+'\n'
#                 resultsFile = open('RobotResults.txt','a+')
#                 resultsFile.write(outString)
#                 resultsFile.close()
#                 epsilon = epsilon + epsilonIncr
#             sigma = sigma + sigmaIncr
#         beta = beta + betaIncr
#     mu = mu + muIncr


 

## Bayesian Optimization of Hyperparameters

In [4]:
import numpy as np

Create a function of the parameters that returns the F score

In [5]:
def entropy_F1(beta=14,sigma=145,mu=0.6,muIncr=0.05,epsilon=35,epsilonIterate=0):
    resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, 
                                  beta, 
                                  sigma, 
                                  mu, 
                                  muIncr,
                                  epsilon,
                                  epsilonIterate,
                                  runClusterMetrics, 
                                  runFinalMetrics, 
                                  createFinalJoin, 
                                  hasHeader)
    return resultsList[9]

In [6]:
entropy_F1(mu=0.5,muIncr=0.05,beta=33,sigma=449,epsilon=31.8,epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.5
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 1873
NewMatrix Iterateblocks Total Time = 1.9605259010568261


>>>>>>>>>>>>
Starting Iteration mu= 0.55
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 1520
NewMatrix Iterateblocks Total Time = 1.5902154007926583


>>>>>>>>>>>>
Starting Iteration mu= 0.6000000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 929
NewMatrix Iterateblocks Total Time = 1.3339230408892035


>>>>>>>>>>>>
Starting Iteration mu= 0.6500000000000001
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 256
NewMatrix Iterateblocks Total Time = 0.6163202654570341


>>>>>>>>>>>>
Starting Iteration mu= 0.7000000000000002
Starting Iteration epsilon= 31.8
Running NewMatrix Jar!

0.73495788

Create functions for variable transformation.

In [7]:
#Softplus to transform real domain into positive
def softplus(x):
    return np.log(1+np.exp(x))

def inv_softplus(x):
    return np.log(np.exp(x)-1)

In [8]:
inv_softplus(1)

0.541324854612918

In [9]:
#Logit to transform real domain into 0-1
def logit(x):    
    return np.log(x+1e-54)-np.log(1-x+1e-54)

def inv_logit(x):
    res=np.exp(x)/(np.exp(x)+1)-2*1e-54
    if res<1e-54:
        res=0
    return res

Import BO library:

In [10]:
from bayes_opt import BayesianOptimization

Create dictionary bounds for the parameters:

In [11]:
import os, sys

In [12]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [13]:
with HiddenPrints():
    print("This will not be printed")

print("This will be printed as before")

This will be printed as before


In [14]:
#Take parameters in transformed space and return f score in transformed space
def entropy_F1_trx(beta_sp,sigma_sp,mu_logit,muIncr_logit,epsilon_sp,epsilonIterate_logit=logit(0)):
    beta_pos=int(softplus(beta_sp))
    sigma_pos=int(softplus(sigma_sp))
    mu_01=inv_logit(mu_logit)
    muIncr_01=inv_logit(muIncr_logit)
    epsilon_pos=softplus(epsilon_sp)
    epsilonIterate_01=inv_logit(epsilonIterate_logit)
    
    print("Mu:%.2f|Mu+:%.2f|Beta:%.2f|Sigma:%.2f|Epsilon:%.2f|EpIter:%.2f" % (mu_01, muIncr_01, beta_pos, sigma_pos, epsilon_pos, epsilonIterate_01))
    
    with HiddenPrints():
        f1=entropy_F1(beta=beta_pos,
                      epsilon=epsilon_pos,
                      epsilonIterate=epsilonIterate_01,
                      muIncr=muIncr_01,
                      sigma=sigma_pos,
                      mu=mu_01)
    
    return logit(f1)

In [15]:
entropy_F1_trx(beta_sp=inv_softplus(33),
               sigma_sp=inv_softplus(449),
               mu_logit=logit(0.5),
               muIncr_logit=logit(0.05),
               epsilon_sp=inv_softplus(31.8))

Mu:0.50|Mu+:0.05|Beta:33.00|Sigma:449.00|Epsilon:31.80|EpIter:0.00


1.0199244346960437

In [64]:
optimizer=None

In [65]:
pbounds = {'beta_sp': (inv_softplus(1), inv_softplus(700)), 
           'sigma_sp': (inv_softplus(1), inv_softplus(700)),
           'mu_logit': (logit(0.5), logit(0.99)),
           'muIncr_logit': (logit(0.01), logit(0.2)),
           'epsilon_sp': (inv_softplus(1), inv_softplus(700))}#,
           #'epsilonIterate_logit': (logit(0), logit(0.25))}

In [66]:
optimizer = BayesianOptimization(
    f=entropy_F1_trx,
    pbounds=pbounds,
    random_state=1,
)

In [67]:
optimizer.probe(
    params={"beta_sp": inv_softplus(15), 
            "sigma_sp": inv_softplus(135),
            "mu_logit":logit(0.5),
            #'epsilonIterate_logit': logit(0),
            "muIncr_logit":logit(0.05),
            "epsilon_sp":inv_softplus(28)},
    lazy=True,
)

In [68]:
optimizer.maximize(
    init_points=10,
    n_iter=50,
    acq="ucb",
    kappa=0.1,
    alpha=1
)

|   iter    |  target   |  beta_sp  | epsilo... | muIncr... | mu_logit  | sigma_sp  |
-------------------------------------------------------------------------------------
Mu:0.50|Mu+:0.05|Beta:15.00|Sigma:135.00|Epsilon:28.00|EpIter:0.00
|  1        |  1.126    |  15.0     |  28.0     | -2.944    |  0.0      |  135.0    |
Mu:0.80|Mu+:0.01|Beta:292.00|Sigma:103.00|Epsilon:504.38|EpIter:0.00
|  2        | -1.058    |  292.2    |  504.4    | -4.595    |  1.389    |  103.2    |
Mu:0.86|Mu+:0.03|Beta:65.00|Sigma:377.00|Epsilon:130.82|EpIter:0.00
|  3        | -1.483    |  65.13    |  130.8    | -3.486    |  1.823    |  377.4    |
Mu:0.98|Mu+:0.02|Beta:293.00|Sigma:19.00|Epsilon:479.82|EpIter:0.00
|  4        | -2.905    |  293.8    |  479.8    | -3.939    |  4.035    |  19.7     |
Mu:0.66|Mu+:0.06|Beta:469.00|Sigma:139.00|Epsilon:292.43|EpIter:0.00
|  5        |  0.7715   |  469.5    |  292.4    | -2.802    |  0.6451   |  139.1    |
Mu:0.96|Mu+:0.03|Beta:560.00|Sigma:613.00|Epsilon:677.80|

|  40       | -1.365    |  550.8    |  64.74    | -2.243    |  1.663    |  460.1    |
Mu:0.59|Mu+:0.01|Beta:646.00|Sigma:543.00|Epsilon:656.38|EpIter:0.00
|  41       |  0.04557  |  646.0    |  656.4    | -4.536    |  0.359    |  543.3    |
Mu:0.84|Mu+:0.01|Beta:688.00|Sigma:670.00|Epsilon:62.09|EpIter:0.00
|  42       | -1.362    |  688.4    |  62.09    | -4.417    |  1.661    |  670.4    |
Mu:0.98|Mu+:0.02|Beta:82.00|Sigma:486.00|Epsilon:602.11|EpIter:0.00
|  43       | -3.754    |  82.95    |  602.1    | -3.844    |  4.11     |  486.3    |
Mu:0.93|Mu+:0.06|Beta:675.00|Sigma:418.00|Epsilon:20.67|EpIter:0.00
|  44       | -2.332    |  675.7    |  20.67    | -2.754    |  2.658    |  418.6    |
Mu:0.64|Mu+:0.01|Beta:586.00|Sigma:448.00|Epsilon:504.40|EpIter:0.00
|  45       |  0.8792   |  586.7    |  504.4    | -4.559    |  0.5787   |  448.5    |
Mu:0.68|Mu+:0.10|Beta:25.00|Sigma:68.00|Epsilon:660.77|EpIter:0.00
|  46       |  0.06138  |  25.47    |  660.8    | -2.231    |  0.7482   |  

In [69]:
optimizer.max

{'target': 1.1263918322593969,
 'params': {'beta_sp': 14.999999694097633,
  'epsilon_sp': 27.999999999999307,
  'muIncr_logit': -2.9444389791664403,
  'mu_logit': 0.0,
  'sigma_sp': 135.0}}

In [70]:
inv_logit(optimizer.max["target"])

0.75517241

In [71]:
params=optimizer.max["params"]

In [72]:
softplus(params["beta_sp"])

15.0

In [73]:
softplus(params["sigma_sp"])

135.0

In [74]:
inv_logit(params["mu_logit"])

0.5

In [75]:
inv_logit(params["muIncr_logit"])

0.05000000000000002

In [76]:
inv_logit(params["epsilonIterate_logit"])

KeyError: 'epsilonIterate_logit'

In [77]:
softplus(params["epsilon_sp"])

28.0

In [36]:
entropy_F1(mu=0.6,
           muIncr=0.05,
           beta=14,
           sigma=145,
           epsilon=35,
           epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.6
Starting Iteration epsilon= 35
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 1715
NewMatrix Iterateblocks Total Time = 1.3094799518585205

Precision = TP / (TP + FP) ......................... = 0.78797468
Recall = TP / (TP + FN) ............................ = 0.88801712
F-Measure .......................................... = 0.83501006


0.83501006